## Introduction

To fully describe the circuit behavior of a building block, one should at least define a Netlist view and a CircuitModel view. Those views can simply be added to the already existing Layoutview. The advantage is that both the Layout and the Circuit model of your component are defined in one place and you are in control on how the matching between the model and the Layout is done. For example the model can be based on:

1. An analytic expression
2. A measurement file
3. A physical simulation
4. Any scripted function that generates an s-matrix.

In short, the tool will not limit you in terms of the frequency models you can define. 

**note**: there are many resources in the documentation that can help you after this training (but which you do not need now to finish this training):

* [Caphe tutorial](http://docs.lucedaphotonics.com/3.1/tutorials/circuitmodels/index.html)
* [Netlist view guide](http://docs.lucedaphotonics.com/3.1/guides/netlist/index.html)
* [Caphe introduction](http://docs.lucedaphotonics.com/3.1/guides/caphe/index.html)

### Netlist View
The netlist is used to describe the terms, and potentially sub-components and connectivity of a component. For our disk resonator, we want to define the two terms (1 input, 1 output). The CircuitModel view will describe the scattering matrix. 

We start by describing the netlist of the disk resonator. The disk resonator has 2 terms as shown below:

<img src="_images/netlist_disk.png" width=250>

The disk has with two ports (in, out)

This netlist contains all terms and connectivity information that is needed by Caphe to create a circuit. Our component, a disk resonator, does not have any internal components nor connectivity and can just be described by the two terms that connect it to the outside world.

Terms are defined using the `_generate_terms(self, terms)` method in the Netlist view. Let’s see how this is done in practice:

In [ ]:
from technologies import silicon_photonics # Import the standard technology
from ipkiss3 import all as i3

In [ ]:
# Fix paths so that relative imports and plotting work in ipython 
import os, sys
sys.path.append('.')
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 6)

In [ ]:
import numpy as np

class DiskWithNetlist(i3.PCell):
    """A Disk resonator.
    """       
    class Netlist(i3.NetlistView):
        
        def _generate_terms(self, terms):
            terms += i3.OpticalTerm(name="in")
            terms += i3.OpticalTerm(name="out")
            return terms           

Optical terms are added to the terms list using the i3.OpticalTerm function. We can now create an instance of the DiskResonator and query its terms

In [ ]:
my_disk = DiskWithNetlist()
my_disk_netlist = my_disk.Netlist()
print my_disk_netlist.terms

### CircuitModel

The behavior of a linear, passive component can be described using its scattering matrix (S-matrix). For a specific component, the scattering matrix defines the amplitude and phase relationship between input signals in each term and each mode, and output signals in each term and each mode. It is therefore an (n x m, n x m) square matrix as illustrated here.

<img src="_images/smatrix.png" width=400>


Our ring resonator has 2 terms and 1 mode (per term). To model its scattering coefficients, 4 coefficients are therefore needed. If we assume no reflections, the transmission of the ring can be approximated by a Lorentzian function whose properties depend on:

1. the optical length of the disk 
2. the loss inside the disk
3. the transmission in the directional coupler

Without going into more details, we assume the following S matrix for our disk resonator. 

\begin{equation}
\left(\begin{array}{c}
B_{in}\\
B_{out}
\end{array}\right)=\left(\begin{array}{cc}
0 & T\\
T & 0
\end{array}\right)\left(\begin{array}{c}
A_{in}\\
A_{out}
\end{array}\right)
\end{equation}

where $A_x$ denote the incoming wave in each port and $B_x$ the outcoming wave. The transmission $T$ can be calculated as:

\begin{equation}
T=\frac{t_{c}-t_{d}\exp(1j\beta l_{d})}{1-t_{c}t_{d}\exp(1j\beta_d l_{d})}
\end{equation}

where 

- $t_c$: field transmission trough the directional coupler. $t_c=1$ means that no power is coupled to the disk.
- $t_d$: field roundtrip transmission through the disk. $t_d=1$ means the no power is lost in the disk.
- $l_d$: length of the disk.
- $\beta_d = \frac{2\pi}{\lambda} n_{eff_d} $: propagation constant of the disk mode and can readily be calculated using the wavelength and the effective index of the mode in de disk.

Let's now create a PCell with both a Netlist and a CircuitModel to see how this is implemented in Ipkiss.

In [ ]:
class RingModel(i3.CompactModel):
    

    parameters = [
       'disk_roundtrip_transmission',
       'disk_n_eff',
       'coupler_bar_transmission',
       'disk_length'
        ]
    
    terms = [i3.OpticalTerm(name='in', n_modes=1), 
             i3.OpticalTerm(name='out', n_modes=1)]
    
    def calculate_smatrix(parameters, env, S):
        
        beta = 2 * np.pi / env.wavelength * parameters.disk_n_eff
        t = parameters.coupler_bar_transmission - parameters.disk_roundtrip_transmission * np.exp(1j * beta * parameters.disk_length)
        n = 1 - parameters.coupler_bar_transmission * parameters.disk_roundtrip_transmission * np.exp(1j * beta * parameters.disk_length)
        
        S['in', 'out'] = S['out', 'in'] = t / n
        S['in', 'in'] = S['out', 'out'] = 0




class DiskResonator(i3.PCell):
    """A Disk resonator.
    """       
    class Netlist(i3.NetlistView):
        
        def _generate_terms(self, terms):
            terms += i3.OpticalTerm(name="in")
            terms += i3.OpticalTerm(name="out")
            return terms   
        
    class CircuitModel(i3.CircuitModelView):
        """
        A simple disk model
        """
        disk_roundtrip_transmission = i3.NonNegativeNumberProperty(default=0.99, doc="Amplitude transmission after one pass in the disk")
        disk_n_eff = i3.PositiveNumberProperty(default=3.0, doc="Effective index of the disk waveguide")
        coupler_bar_transmission = i3.ComplexNumberProperty(default=0.99, doc="Transmission in the (lossless) directional coupler. What passes through and is not droped to the other channel.")
        disk_length = i3.PositiveNumberProperty(default=10.0, doc="disk physical length")
        
        def _generate_model(self):
            return RingModel(disk_roundtrip_transmission=self.disk_roundtrip_transmission,
                             disk_n_eff=self.disk_n_eff,
                             coupler_bar_transmission=self.coupler_bar_transmission,
                             disk_length=self.disk_length)
            



Let’s have a closer look at our RingModel class. It is a self-contained parametric model that can be defined extrally to the class. Inside the class we define the calculate_smatrix ``calculate_smatrix(parameters, env, S)`` method. The parameters of ``calculate_smatrix`` have the following meaning:

- parameters: A list containing the name of the parameters of the model. As we compile our models there are restrictions on the type of parameters that can be passed to a model that only accept (String, complex and real floats and numpy arrays of complex numbers)
- env: a global object that contains the wavelength
- S: the scatter matrix of the model. 

In order to calculate the scattering matrix between the terms for each mode, calculate_smatrix will be evaluated by Caphe for each wavelength.

In [ ]:
my_disk = DiskResonator()
my_disk_cm =  my_disk.CircuitModel(disk_length=10.0,
                                   disk_roundtrip_transmission=0.90,
                                   coupler_bar_transmission=0.90)

### Running a simulation 

Every correctly defined model can generate a S-Matrix based on an array of wavelengths. 
In what follows we run such a simulation and plot the results. 

In [ ]:
# Create a wavelength array.
wavelengths = np.arange(1.3, 1.6, 0.0001)
# Get a the S-Matrix from the disk.
trans_model = my_disk_cm.get_smatrix(wavelengths=wavelengths)
# Plotting simulation the results.
plt.plot(wavelengths, np.abs(trans_model['in', 'out']) ** 2, label='Through-model')
plt.xlabel("Wavelength ($\mu m$)")
plt.ylabel("Power transmission")
plt.ylim([0, 1.0])
plt.show()

#### Exercise 1:

A directional coupler is a light splitting device with 4 terms and 1 mode (per term). 

<img src="_images/directional_coupler.png" width=250>

To model its scattering coefficients, the modes in the two parallel waveguides can be decomposed into two supermodes,
each having a slightly different effective index. The beating of the two supermodes causes light to
transfer from one waveguide to the other one.

Without going into more details, we assume the following:

\begin{array}{c}
\beta_{1}=\frac{2\pi}{\lambda}n_{eff,1}\\
\beta_{2}=\frac{2\pi}{\lambda}n_{eff,2}\\
\Delta\beta=\frac{2\pi}{\lambda}\Delta n_{eff}\\
\phi=\Delta\beta\cdot L_{DC}
\end{array}
    
Where $\Delta n_{eff}$ is the effective index difference between the two supermodes, and $L_{DC}$ is the length of the directional coupler. $\phi$ is the total phase shift caused by the beating of the two supermodes.
If we order the ports as they are shown in the netlist view (["in1", "in2", "out1", "out2"]), then the scatter matrix looks like this:
\begin{equation}
\left(\begin{array}{c}
B_{in_{1}}\\
B_{in2}\\
B_{out1}\\
B_{out2}
\end{array}\right)=\frac{1}{2}e^{j\beta_{1}L_{DC}}\left(\begin{array}{cccc}
0 & 0 & 1+e^{j\phi} & 1-e^{j\phi}\\
0 & 0 & 1-e^{j\phi} & 1+e^{j\phi}\\
1+e^{j\phi} & 1-e^{j\phi} & 0 & 0\\
1-e^{j\phi} & 1+e^{j\phi} & 0 & 0
\end{array}\right)\left(\begin{array}{c}
A_{in_{1}}\\
A_{in2}\\
A_{out1}\\
A_{out2}
\end{array}\right)
\end{equation}

<font color="red">
Exercice 1: Use the skeleton below and create a PCell called DirectionalCoupler. 
- Define a compact model for a directional coupler
- Add a Netlist view the with four terms of the directional coupler.
- Add a CircuitModelView that implements the model described above. 
- Simulate the component.

In [ ]:
class DirectionalCouplerModel(i3.CompactModel):
    
    parameters = ['n_eff_1', 'n_eff_2', 'coupler_length']
    
    terms = [i3.OpticalTerm(name='in1'), 
             i3.OpticalTerm(name='out1'),
            i3.OpticalTerm(name='in2'),
            i3.OpticalTerm(name='out2')]
    

    def calculate_smatrix(parameters, env, S):

        
        straight_trans = 0 # Adapt the model to reflect the correct straight transmission
        cross_trans = 0 # Adapt the model to reflect the correct cross_transmission
        reflection = 0 # No reflections
        cross_talk = 0 # No crosstalk
        
        
        
        S["in1", "out1"] =  S["out1", "in1"] =  S["in2", "out2"] =  S["out2", "in2"] = straight_trans
        S["in1", "out2"] =  S["out2", "in1"] =  S["in2", "out1"] =  S["out1", "in2"] = cross_trans
        S["in1", "in1"] =  S["out2", "out2"] =  S["in2", "in2"] =  S["out1", "out1"] = reflection
        S["in1", "in2"] =  S["in2", "in1"] =  S["out1", "out2"] =  S["out2", "out1"] = cross_talk
        


class DirectionalCoupler(i3.PCell):
    """A directional coupler. We are only interested in simulation behavior 
    here, so we only define a Netlist and Circuit Model.
    """
    # We define our netlist it is made of four ports.  
    class Netlist(i3.NetlistView):
        """
        Netlist view of the directional coupler.
        """
        def _generate_terms(self, terms):
            terms += i3.OpticalTerm(name="in1")
            terms += i3.OpticalTerm(name="in2")
            terms += i3.OpticalTerm(name="out1")
            terms += i3.OpticalTerm(name="out2")
            return terms

    class CircuitModel(i3.CircuitModelView):
        """
        CircuitModelView of the directional coupler.
        """
        n_eff_1 = i3.PositiveNumberProperty(default=3.0, doc="Effective index of the even supermode of the directional coupler.")
        n_eff_2 = i3.PositiveNumberProperty(default=2.99, doc="Effective index of the odd supermode of the directional coupler.")
        coupler_length = i3.PositiveNumberProperty(default=5.0, doc="length of the directional coupler")
        
        def _generate_model(self):
            
            return DirectionalCouplerModel(n_eff_1=self.n_eff_1,
                                            n_eff_2=self.n_eff_2,
                                            coupler_length=self.coupler_length)

                
        


<font color="black">
Solution: [directional_coupler.py](/edit/base_pcell_views_properties/support_files/caphemodelview
/directional_coupler.py)

Simulate here

In [ ]:
my_dc = DirectionalCoupler()
my_dc_cm = my_dc.CircuitModel(n_eff_1=3.0, n_eff_2=2.60, coupler_length=10.0)

# Create a wavelength array.
wavelengths = np.arange(1.3, 1.6, 0.0001)
# Get a the S-Matrix from the disk.
trans_model = my_dc_cm.get_smatrix(wavelengths=wavelengths)
# Plotting simulation the results.
plt.plot(wavelengths, np.abs(trans_model['in1', 'out1']) ** 2, label='Through-model')
plt.plot(wavelengths, np.abs(trans_model['in1', 'out2']) ** 2, label='Drop-model')
plt.xlabel("Wavelength ($\mu m$)")
plt.ylabel("Power transmission")
plt.legend()
plt.ylim([0, 1.0])
plt.show()

Running a circuit simulation of a single component is of course a trivial exercise. In the next notebook you will learn to create your own circuits. 

<div>
<div style="width: 20%; display: inline; margin:0; float: left"> <p> 
<a href="04-picazzo-place-and-autoroute.ipynb"> <img src="_images/annotated_circuit.png" width=200 ></a> </p> </div>
<div style="width: 49%;  display: inline; float: right; margin:0"> <p> <a href="05-picazzo-place-and-autoroute.ipynb"> Training Part 5: Creating simple circuits</a> </p> </div>
</div>